In [6]:

import strawberryfields as sf
from strawberryfields.ops import *
import numpy as np

M = 4
# Create a random complex matrix
random_matrix = np.random.rand(M, M) + 1j * np.random.rand(M, M)

# Perform QR decomposition to get a unitary matrix
q, r = np.linalg.qr(random_matrix)

# Normalize to ensure unitary property (q should already be unitary, but we'll double-check)
unitary_matrix = q / np.linalg.norm(q, axis=0)

boson_sampling_triangular = sf.Program(4)

with boson_sampling_triangular.context as q:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac     | q[2]
    Fock(1) | q[3]


    Interferometer(unitary_matrix, mesh = 'triangular', tol = 1e-10) | q

boson_sampling_triangular.compile(compiler="fock")
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 5})
results = eng.run(boson_sampling_triangular)
# extract the joint Fock probabilities
probs_triangular = results.state.all_fock_probs()

#now try with mesh = 'rectangular'
boson_sampling_rectangular = sf.Program(4)

with boson_sampling_rectangular.context as q:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac     | q[2]
    Fock(1) | q[3]

    Interferometer(unitary_matrix, mesh = 'rectangular', tol = 1e-10) | q
    
boson_sampling_rectangular.compile(compiler="fock")
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 5})
results = eng.run(boson_sampling_rectangular)
# extract the joint Fock probabilities
probs_rect = results.state.all_fock_probs()

print(f"Probability to measure |1 1 0 1> for triangular mesh: {probs_triangular[1,1,0,1]}")
print(f"Probability to measure |1 1 0 1> for triangular mesh: {probs_rect[1,1,0,1]}")

Probability to measure |1 1 0 1> for triangular mesh: 0.0493711617467613
Probability to measure |1 1 0 1> for triangular mesh: 0.16372872401750058


In [ ]:

# code to test the order in t_list and to reconstruct the unitary matrix from the decomposition

def create_T_mn(n, m, theta, phi, n_max):

    T = np.identity(n_max, dtype  = np.complex128)
    T[n][n] = np.exp(1j*phi)*np.cos(theta)
    T[n][m] = -np.sin(theta)
    T[m][n] = np.exp(1j*phi)*np.sin(theta)
    T[m][m] = np.cos(theta)

    return T

def create_inverse_T_mn(n, m, theta, phi, n_max):

    	return create_T_mn(n, m, theta, -phi, n_max).T


N = 5
# Create a random complex matrix
random_matrix = np.random.rand(N, N) + 1j * np.random.rand(N, N)

# Perform QR decomposition to get a unitary matrix
q, r = np.linalg.qr(random_matrix)

# Normalize to ensure unitary property (q should already be unitary, but we'll double-check)
unitary_matrix = q / np.linalg.norm(q, axis=0)

t_list_rectangular, D_rectangular, _ = sf.decompositions.rectangular_phase_end(unitary_matrix) #rectangular_phase_end decomposition
t_list_triangular, D_triangular, _ = sf.decompositions.triangular(unitary_matrix) #triangular decomposition

Ts_rectangular = []
Ts_triangular = []

for t_list in t_list_rectangular:
    Ts_rectangular.append(create_T_mn(*t_list))


#reconstructed_matrix = np.diag(D_rectangular)
reconstructed_matrix = np.identity(N, dtype = np.complex128)

for i in range(len(Ts_rectangular)):
    reconstructed_matrix = Ts_rectangular[i] @ reconstructed_matrix

reconstructed_matrix = np.diag(D_rectangular) @ reconstructed_matrix

print(np.allclose(reconstructed_matrix, unitary_matrix)) #is true

#now triangular
for t_list in t_list_triangular:
    Ts_triangular.append(create_inverse_T_mn(*t_list))

reconstructed_matrix_triangular = np.diag(D_triangular)

for i in range(len(Ts_rectangular)):
    reconstructed_matrix_triangular = Ts_triangular[i] @ reconstructed_matrix_triangular
 
 #note that we need T_inverse for the triangular decomposition. Since T = BS(theta, 0) @ PS(phi) inverse would be  PS(-phi) @ BS(-theta, 0) when appyling them
 #  but in source code for interferometer this is not done!

print(np.allclose(reconstructed_matrix_triangular, unitary_matrix)) #is true now too